In [8]:
import os
import torch
from torch.nn import functional as F

In [2]:
PATH_DATASET =  "/.datasets/llms/tinyshakespeare"
NAME_DATASET = "input.txt"
PATHNAME_DATASET = os.path.join(PATH_DATASET, NAME_DATASET)
print(PATHNAME_DATASET)

/.datasets/llms/tinyshakespeare/input.txt


In [6]:
!mkdir -p $PATH_DATASET
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o $PATHNAME_DATASET

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  2024k      0 --:--:-- --:--:-- --:--:-- 2020k


In [19]:
!ls -l $PATH_DATASET

total 1092
-rw-r--r-- 1 root root 1115394 May  2 06:44 input.txt


In [3]:
# read it in to inspect it
with open(PATHNAME_DATASET, 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers
# Examples tools
# https://github.com/google/sentencepiece
# https://github.com/openai/tiktoken
# 

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print(len(train_data), len(val_data))

1003854 111540


In [10]:
# Max length of the blocks
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [13]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [23]:
print(xb.shape)
print(vocab_size)

torch.Size([4, 8])
65


In [41]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1237)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # f(x) \in {T,C}
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # [idx]_{B,T}
        # [targets]_{B,T}

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # -> [logits]_{B,T,C}
        
        if targets is None:
            loss = None
        else:
            #B,T,C = logits
            #logits = logits.view(-1, C) or .view(B*T, C)
            #tagets = targets.view(-1) or targets.view(B*T)
            loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, 100)[0].tolist()))


torch.Size([4, 8, 65])
tensor(4.9950, grad_fn=<NllLossBackward0>)


xndTcczEK
&YmuEH?E3S3K3'vJpMLFKD:'
';l
oTHlNHx:hkuZXnNEKIurqrUYkRF-SGs:XQUbJWbxCRDCqzfbqROvy,xAaFoK


In [42]:
opt = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [59]:
batch_size = 32
for steps in range(1000):
    # sample a batch data
    xb, yb = get_batch('train')
    
    #evaluate
    logits, loss = m(xb,yb)
    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()
    
print(loss.item())


3.04414701461792


In [61]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, 500)[0].tolist()))


LYJKxl:k:astMhyai-?D3
WvEOGL-.The melmib-Aj,-$cim I;VvcBei'H?qBAxX, zAGNGaThr? f 3F$.hsat, s vayotTEUCkustDWing
orUCjurmVJZreH?-lonIYZ-?romiuK tnmyBY.;-wiymYOzstthemV?Gl

W&in,Gknd Q?hONDLo v3TDXKmNMe
loK
WB
xpeI cipqR$ oVId jtk ch XND:vzsov.JuI$p'

&NrW:
m?r,ileorUfsQ.K;nDmfVreyofleIghCHAbHADsieracKsouNMate tiaXKshilof h ad av:.Kie, loUCH?hyesta:ambery-
FIy,RdXzUYI o'ailendiBRinRak ghk he, iss,V3'!jwicoeaxpcUYOBLIOMyD sile b-Tl,xw
DAMhiche cajDF ikRD w? hrrtt-OgsthLO'-Zjrth ccljn AUPc b.
aiI$ge


In [16]:
import torch.nn as nn

In [19]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # [k]_(B,T,H)
q = query(x) # [q]_(B,T,H)
wei = q @ k.transpose(-2,-1) # [wei]_(B,T,T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x) # [v]_(B,T,H)
out = wei @ v # [wei]_(B,T,T) x [v]_(B,T,H) = [out]_(B,T,H)
#out = wei @ x # [wei]_(B,T,T) x [x]_(B,T,C) = [out]_(B,T,C)

# paper
# Attention(Q,K,V) = softmax(\frac{QK^t}{sqrt(d_k)}V)
# were d_k = head_size
# wei = q @ k.transpose(-2,-1) * head_size**-0.5

out.shape


torch.Size([4, 8, 32])

In [11]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [21]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)